In [1]:
import pandas as pd
import os

In [2]:
%pwd

'd:\\DL\\Live project\\DL-Inuron_Assignment_2\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\DL\\Live project\\DL-Inuron_Assignment_2'

In [92]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    train_data: Path
    test_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_rotation_range: int
    params_width_shift_range: float 
    params_height_shift_range: float
    params_horizontal_flip: bool

In [71]:
from dl_assignment2.constants import *
from dl_assignment2.constants import *
from dl_assignment2.utils.common import read_yaml, create_directories
import tensorflow as tf

In [93]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    


    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        params = self.params
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            train_data=Path(training.train_data),
            test_data=Path(training.test_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_rotation_range=params.ROTATION_RANGE,
            params_width_shift_range=params.WIDTH_SHIFT_RANGE,
            params_height_shift_range=params.HEIGHT_SHIFT_RANGE,
            params_horizontal_flip=params.HORIZONTAL_FLIP,
        )

        return training_config

In [99]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization, GlobalAveragePooling2D

In [102]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    # Function to load a CIFAR-10 batch
    def load_cifar10_batch(self,file_path):
        with open(file_path, 'rb') as f:
            data_dict = pickle.load(f, encoding='bytes')
        return data_dict[b'data'], np.array(data_dict[b'labels'])

    def load_data(self,):
        X_train = []
        y_train = []
        for i in range(1, 6):  # Loop through 5 training batches
            data, labels = self.load_cifar10_batch(os.path.join(self.config.train_data, f"data_batch_{i}"))
            X_train.append(data)
            y_train.append(labels)

        # Convert list to numpy array
        X_train = np.vstack(X_train)
        y_train = np.hstack(y_train)

        # Load testing data
        X_test, y_test = self.load_cifar10_batch(self.config.test_data)

        # Reshape images from (num_samples, 3072) to (num_samples, 32, 32, 3)
        X_train = X_train.reshape(-1, 32, 32, 3)
        X_test = X_test.reshape(-1, 32, 32, 3)

        # Normalize pixel values to range [0,1]
        X_train = X_train.astype('float32') / 255.0
        X_test = X_test.astype('float32') / 255.0

        # Convert class labels to one-hot vectors
        y_train = to_categorical(y_train, 100)
        y_test = to_categorical(y_test, 100)
        
        return X_train, X_test, y_train, y_test
    
    @staticmethod
    def save_model(path: Path, model):
        model.save(path)

    def define_model(self):
       # Define the CNN model
        model = Sequential([
                        Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(32, 32, 3)),
                        BatchNormalization(),
                        MaxPooling2D(2, 2),

                        Conv2D(64, (3,3), activation='relu', padding='same'),
                        BatchNormalization(),
                        MaxPooling2D(2, 2),

                        Conv2D(128, (3,3), activation='relu', padding='same'),
                        BatchNormalization(),
                        MaxPooling2D(2, 2),

                        Conv2D(256, (3,3), activation='relu', padding='same'),
                        BatchNormalization(),
                        GlobalAveragePooling2D(),

                        Dense(512, activation='relu'),
                        Dropout(0.5),
                        Dense(100, activation='softmax')  # 100 classes for CIFAR-100
                    ])
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        return model
    
    def model_train(self,model, X_train, X_test, y_train, y_test):
        datagen = ImageDataGenerator(
            rotation_range=self.config.params_rotation_range,
            width_shift_range=self.config.params_width_shift_range,
            height_shift_range=self.config.params_height_shift_range,
            horizontal_flip=self.config.params_horizontal_flip
        )

        datagen.fit(X_train)

        # Train the model
        model.fit(datagen.flow(X_train, y_train, batch_size=self.config.params_batch_size),
                            validation_data=(X_test, y_test),
                            epochs=self.config.params_epochs,
                            verbose=1)

        self.save_model(
            path=self.config.trained_model_path,
            model=model
        )

In [104]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    X_train, X_test, y_train, y_test = training.load_data()
    model = training.define_model()
    training.model_train(model=model, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

except Exception as e:
    raise e

[2025-03-04 19:35:07,769: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-04 19:35:07,787: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-04 19:35:07,788: INFO: common: created directory at: artifacts]
[2025-03-04 19:35:07,790: INFO: common: created directory at: artifacts\training]
Epoch 1/50
782/782 [==============================] - 149s 188ms/step - loss: 1.7516 - accuracy: 0.3769 - val_loss: 2.2806 - val_accuracy: 0.3534
Epoch 2/50
782/782 [==============================] - 134s 172ms/step - loss: 1.4592 - accuracy: 0.4764 - val_loss: 1.6973 - val_accuracy: 0.4530
Epoch 3/50
782/782 [==============================] - 137s 175ms/step - loss: 1.3630 - accuracy: 0.5148 - val_loss: 1.8742 - val_accuracy: 0.3963
Epoch 4/50
782/782 [==============================] - 141s 180ms/step - loss: 1.2977 - accuracy: 0.5377 - val_loss: 1.6957 - val_accuracy: 0.4439
Epoch 5/50
782/782 [==============================] - 137s 175ms/step - loss: 1.2477 -

In [74]:
import matplotlib.pyplot as plt
config = ConfigurationManager()
training_config = config.get_training_config()
training = Training(config=training_config)
X_train, X_test, y_train, y_test = training.load_data()

# Load CIFAR-10 class names
with open("artifacts/data_ingestion/cifar-10-batches-py/batches.meta", 'rb') as f:
    meta = pickle.load(f, encoding='bytes')
class_names = [name.decode('utf-8') for name in meta[b'label_names']]

# Display some images
fig, axes = plt.subplots(2, 5, figsize=(10, 5))
for i, ax in enumerate(axes.flat):
    ax.imshow(X_train[i])
    ax.set_title(class_names[np.argmax(y_train[i])])
    ax.axis('off')

plt.show()

[2025-03-04 15:00:48,376: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-04 15:00:48,386: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-04 15:00:48,390: INFO: common: created directory at: artifacts]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz', 'local_data_file': 'artifacts/data_ingestion/data.tar.gz', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'}, 'training': {'root_dir': 'artifacts/training', 'trained_model_path': 'artifacts/training/model.h5', 'test_data': 'artifacts/data_ingestion/cifar-10-batches-py/test_batch', 'train_data': 'artifacts/data_ingestion/cifar-10-batches-py'}}


BoxKeyError: "'ConfigBox' object has no attribute 'train_data'"

'artifacts\\data_ingestion\\cifar-10-batches-py\\data_batch_1'